# **Validation for Binomial Confidence Interval**

The goal of this notebook is to empirically the correctness of the 2-sided binomial confidence interval.
Blyth and Hutchinson tabulated values for this interval in their paper [Table of Neyman-shortest unbiased confidence intervals for the binomial parameter](https://www.jstor.org/stable/2333308).
We compare the intervals we get with their data.

In [163]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys; sys.path.insert(0, '..')
from binomial_cis import UMAU_lb, UMAU_ub

### **Load in Validation Table**

In [164]:
sheet = '95_T1'
alpha = (100 - int(sheet[:2])) / 100

# these values are from Blyth and Hutchinson's 1960 paper
df = pd.read_excel('2_sided_val_table.xlsx', header=[0,1], index_col=0, sheet_name=sheet)
df.index = np.round(df.index, 1) # make sure indices are rounded to 1 decimal place
df.head()

2          3          4          5          6          7          8   \
       l     u    l     u    l     u    l     u    l     u    l     u    l   
0.0  0.0   0.0  0.0   0.0  0.0   0.0  0.0   0.0  0.0   0.0  0.0   0.0  0.0   
0.1  0.0  50.0  0.0  29.0  0.0  23.0  0.0  18.0  0.0  15.0  0.0  13.0  0.0   
0.2  0.0  75.0  0.0  50.0  0.0  37.0  0.0  31.0  0.0  26.0  0.0  23.0  0.0   
0.3  0.0  83.0  0.0  59.0  0.0  45.0  0.0  38.0  0.0  32.0  0.0  28.0  0.0   
0.4  0.0  87.0  0.0  65.0  0.0  50.0  0.0  42.0  0.0  35.0  0.0  31.0  0.0   

            9        10      
        u    l     u  l   u  
0.0   0.0  0.0   0.0  0   0  
0.1  11.0  0.0  10.0  0   9  
0.2  20.0  0.0  18.0  0  16  
0.3  25.0  0.0  22.0  0  20  
0.4  28.0  0.0  25.0  0  23

### **Create Dataframe with Our Code**

In [165]:
def fill_our_df(df, alpha):
    # make a copy of the df with all values as nan
    my_df = df.copy()
    my_df.loc[:, :] = np.nan

    # fill all values that aren't nans in the original df
    for t in my_df.index:
        # print("t:", t)
        for n in my_df.columns.get_level_values(0).unique():
            # print("n:", n)
            lb_val = df[n]['l'].loc[t]
            
            # only fill if value in original df
            if not np.isnan(lb_val):
                lb, ub = get_lb_ub(t, n, alpha)
                my_df.loc[t, n] = [round(100*lb), round(100*ub)]
    
    return my_df


def get_lb_ub(t_o, n, alpha):
    # check edge cases
    # see "Nonoptimality of Randomized Confidence Sets" by Casella
    if t_o < alpha:
        p_lb, p_ub = 0, 0
    elif t_o > n + 1-alpha:
        p_lb, p_ub = 1, 1
    else:
        # typical case
        p_lb = UMAU_lb(t_o, n, alpha)
        p_ub = UMAU_ub(t_o, n, alpha)
    return p_lb, p_ub

In [166]:
# these values are generated using our code
our_df = fill_our_df(df, alpha)

### **Compare Values**

In [167]:
our_df - df

2         3         4         5         6         7         8        \
       l    u    l    u    l    u    l    u    l    u    l    u    l    u   
0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.4  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0.9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.6  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.7  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.8  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1.9  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.0  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.1  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.2  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.3  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.4  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.5  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.6  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.7  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.8  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2.9  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3.0  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3.1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0   
3.2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0   
3.3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0   
3.4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0   
3.5  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0  0.0  0.0   
3.6  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0 -1.0   
3.7  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0   
3.8  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0   
3.9  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0   
4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0   
4.1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0   
4.2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0   
4.3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0   
4.4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0   
4.5  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0   
4.6  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4.7  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4.8  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4.9  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5.

In [168]:
# this is the value of the maximum absolute discrepency between the tables
# we consider discrepancies on the order of 1.0 or 2.0 to be acceptable
(our_df - df).abs().max().max()

1.0